In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf
from pathlib import Path
from skimage import io

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# import keras library
import keras

# import Sequential from the keras models module
from keras import Sequential

# import Dense, Dropout, Flatten, Conv2D, MaxPooling2D from the keras layers module
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
def image_loader(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    
    return image

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    with open(labels_file_path, "r") as f:
        classes = f.read().split("\n")
       
    images = []
    labels = []
    image_paths = []
    print(classes)
    
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)
            
            for idx in range(len(classes)):
                if classes[idx] in image_name:
                    labels.append(idx)
                    break
                elif len(classes) == idx:
                    labels.append(len(classes))
                    break
        except:
            pass
        
    with open(image_paths_pickle + ".pickle", "wb") as f:
        pickle.dump(image_paths, f)
    
   # assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [4]:
dataset_path = "storage/dataset/JPEGImages/"
label_path = "labels.txt"
batch_size = 128
num_classes = 1
epochs = 12
image_size = (120,72)

In [5]:
X, y = dataset_preprocessing(dataset_path, label_path, image_size, "test_image_pickle")

['bus', 'bicycle', 'car', 'motorbike', 'pedestrian', 'trafficlight', 'trafficsignal', '']


In [6]:
X.shape

(106920, 72, 120, 3)

In [7]:
y = y.reshape(y.shape[0],1)

In [8]:
y.shape

(106920, 1)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

In [10]:
# set model constants
num_classes = 8

# define model as Sequential
model = Sequential()

# first convolutional layer with 32 filters
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(72, 120, 3)))

# add a second 2D convolutional layer with 64 filters
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))

In [11]:
# reduce dimensionality through max pooling
model.add(MaxPooling2D(pool_size=(2, 2)))

# third convolutional layer with 64 filters
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# add dropout to prevent over fitting
model.add(Dropout(0.25))
# necessary flatten step preceeding dense layer
model.add(Flatten())
# fully connected layer
model.add(Dense(32, activation='relu'))

# add additional dropout to prevent overfitting
model.add(Dropout(0.5))

# prediction layers
model.add(Dense(num_classes, activation='sigmoid', name='preds'))

# show model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 70, 118, 8)        224       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 68, 116, 16)       1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 34, 58, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 56, 32)        4640      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 56, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 57344)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)               

In [12]:
model.compile(
    # set the loss as binary_crossentropy
    loss='sparse_categorical_crossentropy',
    # set the optimizer as stochastic gradient descent
    #optimizer= 'adam',
    optimizer=keras.optimizers.SGD(lr=0.01),
    # set the metric as accuracy
    metrics=['accuracy']
)

# mock-train the model using the first ten observations of the train and test sets
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    verbose=1,
    validation_data=(X_test, y_test)
)

Train on 80190 samples, validate on 26730 samples
Epoch 1/12
80190/80190 [==============================] - 286s 4ms/step - loss: 1.5334 - accuracy: 0.5417 - val_loss: 1.4073 - val_accuracy: 0.5484
Epoch 2/12
80190/80190 [==============================] - 294s 4ms/step - loss: 2.6326 - accuracy: 0.5302 - val_loss: 2.4316 - val_accuracy: 0.5484
Epoch 3/12
80190/80190 [==============================] - 296s 4ms/step - loss: 3.2056 - accuracy: 0.5275 - val_loss: 2.4316 - val_accuracy: 0.5484
Epoch 4/12
80190/80190 [==============================] - 297s 4ms/step - loss: 3.5704 - accuracy: 0.5150 - val_loss: 2.4316 - val_accuracy: 0.5484
Epoch 5/12
80190/80190 [==============================] - 295s 4ms/step - loss: 5.3833 - accuracy: 0.4640 - val_loss: 2.4316 - val_accuracy: 0.5484
Epoch 6/12
80190/80190 [==============================] - 297s 4ms/step - loss: 5.3192 - accuracy: 0.4696 - val_loss: 2.4316 - val_accuracy: 0.5484
Epoch 7/12
80190/80190 [==============================] - 297s

In [13]:
# evaluate model on test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.431586436105051
Test accuracy: 0.5483726263046265
